In [1]:
from torchtext.data.utils import get_tokenizer
from torchtext.datasets import WikiText103
from word2vec import get_word_map

train_dataset, valid_dataset, test_dataset = WikiText103(
    root=".data", split=("train", "valid", "test")
)
tokenizer = get_tokenizer("basic_english")
word_map = get_word_map(train_dataset, tokenizer, 1_801_350)

100%|██████████| 1801350/1801350 [00:37<00:00, 47948.25it/s]


In [4]:
import torch
import time
from word2vec import Word2Vec

EPOCH = 1
CHECKPOINT_PATH = "model.pt"


model = Word2Vec(word_map)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

for epoch in range(EPOCH):
    # optimizer.zero_grad()
    # loss = 0

    for i, sentence in enumerate(train_dataset):
        optimizer.zero_grad()
        loss = 0
        
        words = tokenizer(sentence)
        if len(words) == 0:
            continue

        start = time.time()
        for word_index, word in enumerate(words):
            output = model(words, word_index)
            label = model.one_hot(word)
            loss += criterion(output, label)
        end = time.time()

        print(f"epoch {epoch} [{i}] took {end - start:1.3f}s with {len(words)} words - loss {loss:1.3f}")
        
        loss.backward()
        optimizer.step()

        if i == 100:
            break
    
    # loss.backward()
    # optimizer.step()

print("done training, saving model")

torch.save({
    'epoch': EPOCH,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,
}, CHECKPOINT_PATH)


/Users/hanch/code/personal/nlp-study/.venv/lib/python3.11/site-packages/torch/utils/data/datapipes/iter/combining.py:333: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


epoch 0 [1] took 0.080s with 5 words - loss 61.660
epoch 0 [3] took 2.126s with 123 words - loss 1516.813
